In [1]:
import tensorflow as tf
import numpy as np
import tqdm
from collections import Counter

import os

/home/zhjj/anaconda2/envs/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class Config():
    def __init__(self,word_embedding):
        self.paragraph_encoding_hidden_size=128
        self.paragraph_encoding_layer_num=3
        self.dropout=0.3
        self.question_encoding_hidden_size=128
        self.question_encoding_layer_num=3
        self.word_embedding=word_embedding
        self.batch_size=100

In [3]:
class DocReader():
    def __init__(self,config):
        self.config=config
        self.add_placeholder()
        self.add_word_embedding()
        self.logic=self.get_logic()
        self.loss=self.get_loss(self.logic[0],self.logic[1])
        self.train=self.train(self.loss)
        self.predict=self.predict()
        
    def add_placeholder(self):
        self.paragraph=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size,None])
        self.paragraph_length=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size])
        self.paragraph_mask=tf.placeholder(dtype=tf.float32,shape=[self.config.batch_size,None])
        self.paragraph_features=tf.placeholder(dtype=tf.float32,shape=[self.config.batch_size,None,51])
        self.question=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size,None])
        self.question_length=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size])
        self.question_mask=tf.placeholder(dtype=tf.float32,shape=[self.config.batch_size,None])
        self.start_index=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size])
        self.end_index=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size])
#         self.paragraph=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size,52])
#         self.paragraph_length=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size])
#         self.paragraph_mask=tf.placeholder(dtype=tf.float32,shape=[self.config.batch_size,52])
#         self.paragraph_features=tf.placeholder(dtype=tf.float32,shape=[self.config.batch_size,52,51])
#         self.question=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size,9])
#         self.question_length=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size])
#         self.question_mask=tf.placeholder(dtype=tf.float32,shape=[self.config.batch_size,9])
#         self.start_index=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size])
#         self.end_index=tf.placeholder(dtype=tf.int32,shape=[self.config.batch_size])
                


    
    def add_word_embedding(self):
        self.word_embedding=tf.get_variable('word_embedding',initializer=self.config.word_embedding)
        
        
    def get_logic(self):
        p_encoding=self.paragraph_encoding()
#         print(p_encoding.shape)
        q_encoding=self.question_encoding()
#         print(q_encoding.shape)
        with tf.variable_scope('start'):
            start_scores=self.biliner_attention(p_encoding,q_encoding,self.paragraph_mask)
        with tf.variable_scope('end'):
            end_scores=self.biliner_attention(p_encoding,q_encoding,self.paragraph_mask)
        return start_scores,end_scores
        
        
    def get_loss(self,start_scores,end_scores):
        loss=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.start_index,logits=start_scores)+\
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.end_index,logits=end_scores)
        loss=tf.reduce_mean(loss)
        return loss
        
        
    def train(self,loss):
        optimizer = tf.train.AdamOptimizer()
        train_op=optimizer.minimize(loss)
        return train_op
    
    
    def paragraph_encoding(self):
        with tf.variable_scope('paragraph_encoding'):
            def get_cell():
                cell=tf.nn.rnn_cell.BasicLSTMCell(self.config.paragraph_encoding_hidden_size)
                cell=tf.nn.rnn_cell.DropoutWrapper(cell,output_keep_prob=self.config.dropout)
                return cell
#             cell=tf.nn.rnn_cell.BasicLSTMCell(self.config.paragraph_encoding_hidden_size)
#             cell_1=tf.nn.rnn_cell.BasicLSTMCell(self.config.paragraph_encoding_hidden_size)
#             cell_2=tf.nn.rnn_cell.BasicLSTMCell(self.config.paragraph_encoding_hidden_size)
    #         cell=tf.nn.rnn_cell.DropoutWrapper(cell,output_keep_prob=self.config.dropout)
            cell_fw=tf.nn.rnn_cell.MultiRNNCell([get_cell() for _ in range(self.config.paragraph_encoding_layer_num)])
            cell_bw=tf.nn.rnn_cell.MultiRNNCell([get_cell() for _ in range(self.config.paragraph_encoding_layer_num)])
            paragraph=tf.nn.embedding_lookup(self.word_embedding,self.paragraph)
#             print(paragraph.shape)
            p_es=[paragraph]
            p_es.append(self.paragraph_features)
#             print(self.paragraph_features.shape)
            p_q_scores=self.p_q_aligned()
#             print(p_q_scores.shape)
            p_es.append(p_q_scores)
            paragraph=tf.concat(p_es,-1)
#             print(paragraph.shape)
            p_encoding,_=tf.nn.bidirectional_dynamic_rnn(cell_fw,cell_bw,paragraph,sequence_length=self.paragraph_length,dtype=tf.float32)
#             p_encoding,_=tf.nn.bidirectional_dynamic_rnn(cell_1,cell_2,paragraph,sequence_length=self.paragraph_length,dtype=tf.float32)

            p_encoding=tf.concat(p_encoding,-1)
            
           
            return p_encoding
    
    def question_encoding(self):
        with tf.variable_scope('question_encoding'):
            def get_cell():
                cell=tf.nn.rnn_cell.BasicLSTMCell(self.config.question_encoding_hidden_size)
                cell=tf.nn.rnn_cell.DropoutWrapper(cell,output_keep_prob=self.config.dropout)
                return cell
#             cell_1=tf.nn.rnn_cell.BasicLSTMCell(self.config.question_encoding_hidden_size)
#             cell_2=tf.nn.rnn_cell.BasicLSTMCell(self.config.question_encoding_hidden_size)
#         cell=tf.nn.rnn_cell.DropoutWrapper(cell,output_keep_prob=self.config.dropout)
            cell_fw=tf.nn.rnn_cell.MultiRNNCell([get_cell() for _ in range(self.config.question_encoding_layer_num)])
            cell_bw=tf.nn.rnn_cell.MultiRNNCell([get_cell() for _ in range(self.config.question_encoding_layer_num)])
            question=tf.nn.embedding_lookup(self.word_embedding,self.question)
            q_encoding,_=tf.nn.bidirectional_dynamic_rnn(cell_fw,cell_bw,question,sequence_length=self.question_length,dtype=tf.float32)
#             q_encoding,_=tf.nn.bidirectional_dynamic_rnn(cell_1,cell_2,question,sequence_length=self.question_length,dtype=tf.float32)
            q_encoding=tf.concat(q_encoding,-1)
            alpha=self.liner_attention(q_encoding,self.question_mask)
            alpha=tf.concat([tf.expand_dims(alpha,-1)]*self.config.question_encoding_hidden_size*2,-1)
            q_encoding=tf.reduce_sum(tf.multiply(q_encoding,alpha),axis=1)
            return q_encoding
    
    def liner_attention(self,x,x_mask):
        w=tf.get_variable('liner_attention_w',shape=[x.shape[-1],1])
        scores=tf.matmul(tf.reshape(x,(-1,x.shape[-1])),w)
        scores=tf.add(tf.reshape(scores,(x.shape[0],-1)),x_mask)
        alpha=tf.nn.softmax(scores)
        return alpha
    
    def biliner_attention(self,x,y,x_mask):
        w=tf.get_variable('biliner_attention_w',shape=[y.shape[-1],x.shape[-1]])
        yw=tf.matmul(y,w)
        yw=tf.expand_dims(yw,axis=-1)
        scores=tf.matmul(x,yw)
        scores=tf.reshape(scores,[x.shape[0],-1])
        return tf.add(scores,x_mask)
    
    
    def p_q_aligned(self):
        question=tf.nn.embedding_lookup(self.word_embedding,self.question)
        paragraph=tf.nn.embedding_lookup(self.word_embedding,self.paragraph)
        q_shape=question.shape
        p_shape=paragraph.shape
        w1=tf.get_variable('p_q_aligned_w1',shape=[q_shape[-1],q_shape[-1]])
        w2=tf.get_variable('p_q_aligned_w2',shape=[p_shape[-1],p_shape[-1]])
        question=tf.nn.relu(tf.reshape(tf.matmul(tf.reshape(question,[-1,question.shape[-1]]),w1),[q_shape[0],-1,q_shape[-1]]))
        paragraph=tf.nn.relu(tf.reshape(tf.matmul(tf.reshape(paragraph,[-1,paragraph.shape[-1]]),w2),[p_shape[0],-1,p_shape[-1]]))
        scores=tf.matmul(paragraph,tf.transpose(question,perm=[0,2,1]))
#         scores=tf.add(scores,self.question_mask)
        scores=tf.nn.softmax(scores)
        scores=tf.matmul(scores,question)
        
        return scores
    
    def predict(self):
        pre_s=tf.nn.softmax(self.logic[0])
        pre_e=tf.nn.softmax(self.logic[1])
        scores=tf.matmul(tf.expand_dims(pre_s,-1),tf.expand_dims(pre_e,1))
        scores_=tf.reshape(scores,[pre_s.shape[0],-1])
        index=tf.argmax(scores_,axis=-1)
#         pre_s=tf.div(index,pre_s.shape[1])
#         pre_e=tf.mod(index,pre_s.shape[1])
        return index,scores
        
        
        
        
        
        
        
        
        
    

In [4]:
class Data():
    def __init__(self,train_fname,val_fname,vocab_size,word_embedding_fname,word_embedding_size):
        self.counter=Counter()
        self.vocabulary=['**unknow**','**null**']
        self.feature_dict={}
        self.vocab_size=vocab_size
        self.word_vecs={}
        self.word_embedding_size=word_embedding_size
        self.data_index={'train':0,
                        'val':0}
        self.word_to_idx={}
        self.create(train_fname,val_fname,vocab_size,word_embedding_fname)
        
    
    
    def load_data(self, filename):
        """Load examples from preprocessed file.
        One example per line, JSON encoded.
        """
        print('load data from %s\n' % filename)
        # Load JSON lines
        with open(filename) as f:
            examples = [eval(line) for line in f]
        for ex in tqdm.tqdm(examples):
            ex['question'] = [w.lower() for w in ex['question']]
            self.counter.update(ex['question'])
            ex['document'] = [w.lower() for w in ex['document']]
            self.counter.update(ex['document'])
        return examples
  

    def build_feature_dict(self,examples):
        """Index features (one hot) from fields in examples and options."""
        print('build_feature_dict.............\n')
        def _insert(feature):
            if feature not in self.feature_dict:
                self.feature_dict[feature] = len(self.feature_dict)
        
        # Exact match features
        _insert('in_question_uncased')
        
        # Part of speech tag features 
        for ex in examples:
            for w in ex['pos']:
                _insert('pos=%s' % w)

        # Named entity tag features
        for ex in examples:
            for w in ex['ner']:
                _insert('ner=%s' % w)

        # Term frequency feature
        _insert('tf')
        print('build_feature_dict:{}'.format(self.feature_dict))
      
    def get_vocabulary(self,vocab_size):
        vocab_counter=self.counter.most_common()
        v_size=vocab_size-2
        assert v_size<=len(vocab_counter), 'vocabulary_size:%d is too large' % vocab_size
        v_count=0.0
        
        for i in range(v_size):
            self.vocabulary.append(vocab_counter[i][0])
            v_count+=vocab_counter[i][1]
        v_all=v_count
        for i in range(v_size,len(vocab_counter)):
            v_all+=vocab_counter[i][1]
        print('vocabulary of the total ratio is %f' % (v_count/v_all))    
            
    def load_word_embedding(self,fname):
        if os.path.isfile(fname):
            print('loading word_embedding form {}'.format(fname))
            with open(fname) as f:
                for line in tqdm.tqdm(f):
                    line_values=line.strip().split()
                    word=line_values[0].lower()
                    assert self.word_embedding_size==len(line_values[1:]), '%s embedding_size!=self.word_embedding_size' % fname
                    if word in self.vocabulary:
                        self.word_vecs[word]=np.array(line_values[1:],dtype='float32')
   
    def add_word_embedding(self,fname):
        print('add word_embedding not in %s' % fname)
        for word in tqdm.tqdm(self.vocabulary):
            if word not in self.word_vecs:
                self.word_vecs[word]=np.random.uniform(-1,1,self.word_embedding_size)
                    
    def get_word_embedding(self):
        self.word_embedding=np.zeros(shape=(self.vocab_size,self.word_embedding_size),dtype='float32')
        for i in range(len(self.vocabulary)):
            self.word_embedding[i]=self.word_vecs[self.vocabulary[i]]
            
    
    def add_word_to_idx(self):
        for i in range(len(self.vocabulary)):
            self.word_to_idx[self.vocabulary[i]]=i
    
    
    def get_idx_from_sent(self,sent):
        x=[]
        for word in sent:
            if word in self.word_to_idx:
                x.append(self.word_to_idx[word])
            else:
                x.append(self.word_to_idx['**unknow**'])
        return x
    
    def vectorize(self,ex):
        """Torchify a single example."""

        # Index words
        document = self.get_idx_from_sent(ex['document'])
        question = self.get_idx_from_sent(ex['question'])

        # Create extra features vector
        
        features = np.zeros([len(ex['document']),len(self.feature_dict)])
        

        # f_{exact_match}
        for i in range(len(ex['document'])):
            if ex['document'][i] in ex['question']:
                features[i][self.feature_dict['in_question_uncased']] = 1.0
                
        # f_{token} (POS)
        for i, w in enumerate(ex['pos']):
            f = 'pos=%s' % w
            if f in self.feature_dict:
                features[i][self.feature_dict[f]] = 1.0

        # f_{token} (NER)
        for i, w in enumerate(ex['ner']):
            f = 'ner=%s' % w
            if f in self.feature_dict:
                features[i][self.feature_dict[f]] = 1.0

        # f_{token} (TF)
        counter = Counter(ex['document'])
        l = len(ex['document'])
        for i, w in enumerate(ex['document']):
            features[i][self.feature_dict['tf']] = counter[w] * 1.0 / l

        # Maybe return without target
        if 'answers' not in ex:
            return document, features, question, ex['id']

        
        start = ex['answers'][0][0]
        end = ex['answers'][0][1]

        return document, features, question, start, end, ex['id']
    
    
    def process_batch(self,batch):
        """Gather a batch of individual examples into one batch."""
        NUM_INPUTS = 3
        NUM_TARGETS = 2
        NUM_EXTRA = 1

        ids = [ex[-1] for ex in batch]
        docs = [ex[0] for ex in batch]
        features = [ex[1] for ex in batch]
        questions = [ex[2] for ex in batch]

        # Batch documents and features
        docs_len=[len(d) for d in docs]
        max_length = max(docs_len)
        x1 = np.ones([len(batch),max_length])
        x1_mask = np.full([len(batch), max_length],-np.inf)
        x1_f = np.zeros([len(batch), max_length, features[0].shape[1]])
        for i, d in enumerate(docs):
            x1[i, :len(d)]=d
            x1_mask[i, :len(d)]=[0]*len(d)
            for j in range(len(d)):
                x1_f[i,j,:]=features[i][j]

        # Batch questions
        ques_len=[len(q) for q in questions]
        max_length = max(ques_len)
        x2 = np.ones([len(questions), max_length])
        x2_mask = np.full([len(questions), max_length],-np.inf)
        for i, q in enumerate(questions):
            x2[i, :len(q)]=q
            x2_mask[i, :len(q)]=[0]*len(q)

        # Maybe return without targets
        if len(batch[0]) == NUM_INPUTS + NUM_EXTRA:
            return x1, x1_f, x1_mask, x2, x2_mask, ids

        elif len(batch[0]) == NUM_INPUTS + NUM_EXTRA + NUM_TARGETS:
            y_s = [ex[3] for ex in batch]
            y_e = [ex[4] for ex in batch]
        else:
            raise RuntimeError('Incorrect number of inputs per example.')

        return x1,x1_f,docs_len,x1_mask,x2,ques_len,x2_mask,y_s,y_e,ids
        
        
    def process_data(self,examples):
        dataset=[self.vectorize(ex) for ex in examples]
        return dataset
    
    def get_next_batch(self,dataset,batch_size,data_class):
        data_len=len(dataset)
        batch_num=int(data_len/batch_size)
        if self.data_index[data_class]==batch_num:
            self.data_index[data_class]=0
        c_index=self.data_index[data_class]
        batch_data=dataset[c_index*batch_size:(c_index+1)*batch_size]
        self.data_index[data_class]+=1
        return self.process_batch(batch_data)
             
    def create(self,train_fname,val_fname,vocab_size,word_embedding_fname):
        examples=self.load_data(train_fname)
        self.build_feature_dict(examples)
        self.get_vocabulary(vocab_size)
        self.load_word_embedding(word_embedding_fname)
        self.add_word_embedding(word_embedding_fname)
        self.add_word_to_idx()
        self.train_data=self.process_data(examples)
        examples=self.load_data(val_fname)
        self.val_data=self.process_data(examples)
        self.get_word_embedding()

In [ ]:
data=Data('datas/train.json','datas/valid.json',40000,'datas/word_embeddings_64.txt',64)

In [ ]:
import pickle
data_file=open('datas/data','rb')
data=pickle.load(data_file)
x=data.get_next_batch(data.train_data,2,'train')
for i in x:
    try:
        print(i.shape)
    except:
        continue

In [ ]:

tf.reset_default_graph()
import pickle
data_file=open('datas/data','rb')
data=pickle.load(data_file)
config=Config(data.word_embedding)
epoch=50
model=DocReader(config)
batch_num=int(len(data.train_data)/config.batch_size)
init=tf.global_variables_initializer()
saver=tf.train.Saver()
loss_summary=tf.summary.scalar('loss',model.loss)
print('batch_num:%d' % batch_num)
def get_train_feed_dict(model,data):
    batch_data=data.get_next_batch(data.train_data,config.batch_size,'train')
    return {model.paragraph:batch_data[0],
            model.paragraph_length:batch_data[2],
            model.paragraph_mask:batch_data[3],
            model.paragraph_features:batch_data[1],
            model.question:batch_data[4],
            model.question_length:batch_data[5],
            model.question_mask:batch_data[6],
            model.start_index:batch_data[7],
            model.end_index:batch_data[8]}
with tf.Session() as sess:
    writer=tf.summary.FileWriter('logs',sess.graph)
    sess.run(init)
#     saver.restore(sess,'../models/model')
    for i in range(epoch):
        print('epoch:%d' % i)
        for j in tqdm.tqdm(range(batch_num)):
            feed_dict=get_train_feed_dict(model,data)
            loss_str,_=sess.run([loss_summary,model.train],feed_dict=feed_dict)
            writer.add_summary(loss_str,i*batch_num+j)
#             print(sess.run(model.loss,feed_dict=feed_dict),'------------------------')
        save_path='models/model_'+str(i)       
        save_path=saver.save(sess,save_path)
        print("Model saved in path: %s" % save_path)

In [5]:
def get_answer(index1,index2,p):
    if(index1>index2):
        return 'unknow'
    else:
        return ''.join(p[index1:index2+1])

def val(data,model,sess,config):
    result=[]
    batch_num=int(len(data.val_data)/config.batch_size)
    for _ in tqdm.tqdm(range(batch_num)):
        batch_data=data.get_next_batch(data.val_data,config.batch_size,'val')
        feed_dict={model.paragraph:batch_data[0],
                   model.paragraph_length:batch_data[2],
                   model.paragraph_mask:batch_data[3],
                   model.paragraph_features:batch_data[1],
                   model.question:batch_data[4],
                   model.question_length:batch_data[5],
                   model.question_mask:batch_data[6],
                   model.start_index:batch_data[7],
                   model.end_index:batch_data[8]}
        index,scores=sess.run(model.predict,feed_dict=feed_dict)
        pre_s=sess.run(tf.div(index,scores.shape[1]))
        pre_e=sess.run(tf.mod(index,scores.shape[-1]))
        for i in range(config.batch_size):
            id=batch_data[-1][i]
            p_ind=np.asarray(batch_data[0][i],dtype='int32')
            paragraph=[data.vocabulary[j] for j in p_ind]
            answer=get_answer(pre_s[i],pre_e[i],paragraph)
            result.append((id,answer,scores[i][pre_s[i]][pre_e[i]]))
    return result
        
        
        
    
    
    
    
    
tf.reset_default_graph()
import pickle
data_file=open('datas/data','rb')
data=pickle.load(data_file)
config=Config(data.word_embedding)
epoch=50
model=DocReader(config)
batch_num=int(len(data.train_data)/config.batch_size)
init=tf.global_variables_initializer()
saver=tf.train.Saver()
loss_summary=tf.summary.scalar('loss',model.loss)
print('batch_num:%d' % batch_num)
def get_train_feed_dict(model,data,dataset):
    batch_data=data.get_next_batch(dataset,config.batch_size,'train')
    return {model.paragraph:batch_data[0],
            model.paragraph_length:batch_data[2],
            model.paragraph_mask:batch_data[3],
            model.paragraph_features:batch_data[1],
            model.question:batch_data[4],
            model.question_length:batch_data[5],
            model.question_mask:batch_data[6],
            model.start_index:batch_data[7],
            model.end_index:batch_data[8]}
with tf.Session() as sess:
    writer=tf.summary.FileWriter('logs',sess.graph)
    sess.run(init)
    saver.restore(sess,'models/model_49')
#     feed_dict=get_train_feed_dict(model,data,data.train_data)
#     logic=sess.run(model.logic,feed_dict=feed_dict)
#     pre_s=sess.run(tf.nn.softmax(logic[0]))
#     pre_e=sess.run(tf.nn.softmax(logic[1]))
    result=val(data,model,sess,config)

batch_num:626
INFO:tensorflow:Restoring parameters from models/model_49


100%|██████████| 148/148 [01:20<00:00,  1.84it/s]


In [11]:
print(result[0])
print(result[0][-1])

('15000', '特殊教育学校', 0.99951774)
0.999518


In [13]:
result_dict={}
for i in result:
    id=str(i[0])
    if id in result_dict:
        if result_dict[id][0]<i[-1]:
            result_dict[id]=[i[-1],str(i[1])]
    else:
        result_dict[id]=[i[-1],str(i[1])]

In [14]:
for i in result_dict:
    print(i)
    break

15000


In [15]:
with open('datas/val_predict_.txt','w') as f:
    for i in result_dict:
        line=str(i)+'\t'+str(result_dict[i][1])+'\n'
        f.write(line)

In [ ]:
p0=np.asarray(feed_dict[model.paragraph][0],dtype='int32')
print([data.vocabulary[i] for i in p0])

In [ ]:
print(pre_s.shape)
print(type(pre_s))
with tf.Session() as s:
    pre_s=tf.expand_dims(pre_s,-1)
    pre_e=tf.expand_dims(pre_e,1)
    scores_=s.run(tf.matmul(pre_s,pre_e))
    scores=s.run(tf.reshape(tf.matmul(pre_s,pre_e),(pre_s.shape[0],-1)))
    print(pre_s.shape)
    print(pre_e.shape)
#     print(s.run(scores))
    
    index=tf.argmax(scores,axis=-1)
    print(index.shape)
    print(s.run(index))
    print(s.run(tf.div(index,pre_s.shape[1])))
    print(s.run(tf.mod(index,pre_s.shape[1])))
    print(scores_[0][12][12])

In [ ]:
x=tf.constant([[[1,3,2],[4,6,5]],[[7,8,9],[10,11,12]]])
print(x.shape)
x_=tf.reshape(x,(x.shape[0],-1))
with tf.Session() as sess:
    print(sess.run(tf.argmax(x_,axis=-1)))
    print(sess.run(tf.argmax(x,axis=1)))
    

In [ ]:
    def decode(score_s, score_e, top_n=1, max_len=None):
        """Take argmax of constrained score_s * score_e.

        Args:
            score_s: independent start predictions
            score_e: independent end predictions
            top_n: number of top scored pairs to take
            max_len: max span length to consider
        """
        pred_s = []
        pred_e = []
        pred_score = []
        max_len = max_len or score_s.size(1)
        for i in range(score_s.size(0)):
            # Outer product of scores to get full p_s * p_e matrix
            scores = torch.ger(score_s[i], score_e[i])

            # Zero out negative length and over-length span scores
            scores.triu_().tril_(max_len - 1)

            # Take argmax or top n
            scores = scores.numpy()
            scores_flat = scores.flatten()
            if top_n == 1:
                idx_sort = [np.argmax(scores_flat)]
            elif len(scores_flat) < top_n:
                idx_sort = np.argsort(-scores_flat)
            else:
                idx = np.argpartition(-scores_flat, top_n)[0:top_n]
                idx_sort = idx[np.argsort(-scores_flat[idx])]
            s_idx, e_idx = np.unravel_index(idx_sort, scores.shape)
            pred_s.append(s_idx)
            pred_e.append(e_idx)
            pred_score.append(scores_flat[idx_sort])
        return pred_s, pred_e, pred_score

In [ ]:
tf.reset_default_graph()
import pickle
data_file=open('datas/data','rb')
data=pickle.load(data_file)
config=Config(data.word_embedding)
epoch=50




sess=tf.Sesssion() 
batch_num=int(len(data.train_data)/config.batch_size)
init=tf.global_variables_initializer()
saver=tf.train.Saver()
loss_summary=tf.summary.scalar('loss',model.loss)


def get_train_feed_dict(model,data):
    batch_data=data.get_next_batch(data.train_data,config.batch_size,'train')
    return {paragraph:batch_data[0],
            paragraph_length:batch_data[2],
            paragraph_mask:batch_data[3],
            paragraph_features:batch_data[1],
            question:batch_data[4],
            question_length:batch_data[5],
            question_mask:batch_data[6],
            start_index:batch_data[7],
            end_index:batch_data[8]}
writer=tf.summary.FileWriter('../logs',sess.graph)
sess.run(init)

for i in range(epoch):
    for j in tqdm.tqdm(range(batch_num)):
        feed_dict=get_train_feed_dict(model,data)
        loss_str,_=sess.run([loss_summary,model.train],feed_dict=feed_dict)
        writer.add_summary(loss_str,i*batch_num+j)
        if (j+1)%500==0:
                point=str(i)+'_'+str(j)
                save_path=saver.save(sess,'../models/model_%s' % point)
                print("Model saved in path: %s" % save_path)


        

paragraph=tf.placeholder(dtype=tf.int32,shape=[config.batch_size,None])
paragraph_length=tf.placeholder(dtype=tf.int32,shape=[config.batch_size])
paragraph_mask=tf.placeholder(dtype=tf.float32,shape=[config.batch_size,None])
paragraph_features=tf.placeholder(dtype=tf.float32,shape=[config.batch_size,None,None])
question=tf.placeholder(dtype=tf.int32,shape=[config.batch_size,None])
question_length=tf.placeholder(dtype=tf.int32,shape=[config.batch_size])
question_mask=tf.placeholder(dtype=tf.float32,shape=[config.batch_size,None])
start_index=tf.placeholder(dtype=tf.int32,shape=[config.batch_size])
end_index=tf.placeholder(dtype=tf.int32,shape=[config.batch_size])
    

word_embedding=tf.get_variable('word_embedding',initializer=config.word_embedding)
        
        
def get_logic():
    p_encoding=paragraph_encoding()
    q_encoding=question_encoding()
    start_scores=biliner_attention(p_encoding,q_encoding,paragraph_mask)
    end_scores=biliner_attention(p_encoding,q_encoding,paragraph_mask)
    return start_scores,end_scores
        
        
def get_loss(start_scores,end_scores):
    loss=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=start_index,logits=start_scores)+\
    tf.nn.sparse_softmax_cross_entropy_with_logits(labels=end_index,logits=end_scores)
    loss=tf.reduce_mean(loss)
    return loss
        
        
def train(loss):
    optimizer = tf.train.AdamOptimizer()
    train_op=optimizer.minimize(loss)
    return train_op
    
    
def paragraph_encoding():
    def get_cell():
        cell=tf.nn.rnn_cell.BasicLSTMCell(config.paragraph_encoding_hidden_size)
        cell=tf.nn.rnn_cell.DropoutWrapper(cell,output_keep_prob=self.config.dropout)
        return cell
    
    cell_fw=tf.nn.rnn_cell.MultiRNNCell([get_cell() for _ in range(self.config.paragraph_encoding_layer_num)])
    cell_bw=tf.nn.rnn_cell.MultiRNNCell([get_cell() for _ in range(self.config.paragraph_encoding_layer_num)])
    paragraph=tf.nn.embedding_lookup(word_embedding,paragraph)
    print(paragraph)
    p_encoding,_=tf.nn.bidirectional_dynamic_rnn(cell_fw,cell_bw,paragraph,sequence_length=self.paragraph_length,dtype=tf.float32)
#     p_encoding,_=tf.nn.bidirectional_dynamic_rnn(cell,cell,paragraph,sequence_length=paragraph_length,dtype=tf.float32)

    p_encoding=tf.concat(p_encoding,-1)
    p_es=[p_encoding]
    p_es.append(paragraph_features)
    p_q_scores=p_q_aligned()
    p_es.append(p_q_scores)
    p_encoding=tf.concat(p_es,-1)
    return p_encoding
    
def question_encoding():
    cell=tf.nn.rnn_cell.BasicLSTMCell(config.question_encoding_hidden_size)
    cell=tf.nn.rnn_cell.DropoutWrapper(cell,output_keep_prob=config.dropout)
    cell_fw=tf.nn.rnn_cell.MultiRNNCell([cell]*config.question_encoding_layer_num)
    cell_bw=tf.nn.rnn_cell.MultiRNNCell([cell]*config.question_encoding_layer_num)
    question=tf.nn.embedding_lookup(word_embedding,question)
    q_encoding,_=tf.nn.bidirectional_dynamic_rnn(cell_fw,cell_bw,question,sequence_length=question_length,dtype=tf.float32)
    q_encoding=tf.concat(q_encoding,-1)
    alpha=liner_attention(q_encoding,question_mask)
    alpha=tf.cast([tf.expand_dims(alpha,-1)]*config.question_encoding_hidden_size*2,-1)
    q_encoding=tf.reduce_sum(tf.multiply(q_encoding,alpha),axis=1)
    return q_encoding
    
def liner_attention(x,x_mask):
    w=tf.get_variable('liner_attention_w',shape=[x.shape[-1],1])
    scores=tf.matmul(tf.reshape(x,(-1,x.shape[-1])),w)
    scores=tf.add(tf.reshape(scores,(x.shape[0],x.shape[1])),x_mask)
    alpha=tf.nn.softmax(scores)
    return alpha
    
def biliner_attention(x,y,x_mask):
    w=tf.get_variable('biliner_attention_w',shape=[y.shape[-1],x.shape[-1]])
    yw=tf.matmul(y,w)
    scores=[]
    for i in range(x.shape[0]):
        scores_i=[]
        for j in range(x.shape[1]):
            scores_i.append(tf.reduce_sum(tf.multiply(yw[i],x[i][j])))
    return tf.add(scores,x_mask)
    
    
def p_q_aligned():
    question=tf.nn.embedding_lookup(word_embedding,question)
    paragraph=tf.nn.embedding_lookup(self.word_embedding,self.paragraph)
    q_shape= sess.run(question,feed_dict={question:}).shape
        p_shape=paragraph.eval(self.sess).shape
        w1=tf.get_variable('p_q_aligned_w1',shape=[q_shape[-1],q_shape[-1]])
        w2=tf.get_variable('p_q_aligned_w2',shape=[p_shape[-1],p_shape[-1]])
        question=tf.nn.relu(tf.reshape(tf.matmul(tf.reshape(question,[-1,question.shape[-1]]),w1),[q_shape[0],-1,q_shape[-1]]))
        paragraph=tf.nn.relu(tf.reshape(tf.matmul(tf.reshape(paragraph,[-1,paragraph.shape[-1]]),w2),[p_shape[0],-1,p_shape[-1]]))
        scores=[]
        for i in range(p_shape[0]):
            scores_i=[]
            for j in range(p_shape[1]):
                scores_i_j=[]
                for k in range(q_shape[1]):
                    score=tf.add(tf.reduce_sum(tf.multiply(paragraph[i][j],question[i][k])),\
                                 self.question_mask[i][k])
                    scores_i_j.append(score)
                scores_i.append(scores_i_j)
            scores.append(scores_i)
        scores=tf.nn.softmax(scores)
        return scores
        
        
        
        
        
        
    

In [ ]:
print(x)
# print(x[0])
# print(x[1])
# print(x[2])
# print(x[7])
# print(data.train_data[0])

In [ ]:
import pickle
data_file=open('datas/data','wb')
pickle.dump(data,data_file)

In [ ]:
x=[[1,2],[3,4]]
z=tf.constant([1,2,4])
# z=tf.reshape(z,[1,-1,1])
# print(z.shape)
# z=tf.reduce_sum(z)
z[0]=3
# for i in range(tf.c(z[0])):
#     print(i)
y=[[[1,1],[1,1]],[[2,2],[2,2]]]
r=tf.multiply(tf.concat([tf.expand_dims(x,-1)]*2,-1),y)
with tf.Session() as s:
#     print(s.run(x))
#     print(s.run(r))
#     print(s.run(tf.reduce_sum(y,axis=-1)))
#     print(s.run(z+1))
    print(s.run(z))
    print(s.run(tf.transpose(z[0][1])))

In [ ]:
x=[1.,1.,3.,4.,1.]
y=[1,1,1,-float('inf'),-float('inf')]
with tf.Session() as s:
    print(s.run(tf.multiply(x,y)))
    print(s.run(tf.nn.softmax(tf.multiply(x,y))))

In [ ]:
print(0.000000001*-float('inf'))

In [ ]:
x=np.full([2,3],-np.inf)
print(x.shape[1])
x[0][1]=1
x[0][:2]=[2]*2
x[1][0]=1
x[1,].fill(3)
print(x)
# x=tf.constant(x,dtype=tf.float32)
# y=tf.fill([2,3],9.0)
with tf.Session() as s:
    print(s.run(tf.nn.softmax(x)))
#     print(s.run(tf.reduce_sum(x)))
#     print(s.run(y))

In [ ]:
print(1+\
      2)

In [ ]:
print('1\n2')

In [ ]:
y=np.zeros((2,3,4))
x=np.ones((2,4))
print(y)
y[1,1,:]=x[0]
print(y)

In [ ]:
x={1:1}
x[1]+=1
print(x)

In [ ]:
x=[[[1,2],[2,-np.inf]],[[1,2],[2,2]]]

# y=[[[1,1],[2,2]]]
x_=tf.placeholder(dtype=tf.float32)
def test():
    y=np.ones((2,3))
    y[0][0]=x_[0][0][0]
    return y
with tf.Session() as s:
#     print(s.run(x_+1,feed_dict={x_:x}))
#     print(s.run(tf.concat([x,y],-1)))
    print(s.run(tf.nn.softmax(x_),feed_dict={x_:x}))
    print(s.run(test(),feed_dict={x_:x}))
    print(s.run(x_[0][0][0],feed_dict={x_:x}))

In [ ]:
print(1*-np.inf)

In [ ]:
print(-np.inf*-np.inf)

In [ ]:
tf.reset_default_graph()
x=tf.constant([[[1,1,1],[2,2,2],[3,3,3]]],dtype=tf.float32)

y=tf.constant([[[1,1,1],[2,2,2]]],dtype=tf.float32)
# tf.assign()

x_mask=tf.constant([[0,0,-np.inf]])
y_mask=tf.constant([[0,-np.inf]])

def p_q_aligned(question,paragraph,question_mask,paragraph_mask):
        q_shape=question.shape
        print(q_shape)
        p_shape=paragraph.shape
        print(p_shape)
        w1=tf.get_variable('p_q_aligned_w1',shape=[q_shape[-1],q_shape[-1]])
        w2=tf.get_variable('p_q_aligned_w2',shape=[p_shape[-1],p_shape[-1]])
        question=tf.nn.relu(tf.reshape(tf.matmul(tf.reshape(question,[-1,question.shape[-1]]),w1),q_shape))
        paragraph=tf.nn.relu(tf.reshape(tf.matmul(tf.reshape(paragraph,[-1,paragraph.shape[-1]]),w2),p_shape))
        scores=[]
        for i in range(p_shape[0]):
            scores_i=[]
            for j in range(p_shape[1]):
                scores_i_j=[]
                for k in range(q_shape[1]):
                    score=tf.reduce_sum(tf.multiply(paragraph[i][j],question[i][k]))
#                     score=tf.add(score,paragraph_mask[i][j])
                    score=tf.add(score,question_mask[i][k])
                    scores_i_j.append(score)
                scores_i.append(scores_i_j)
            scores.append(scores_i)
        scores=tf.nn.softmax(scores)
        return scores
    
result=p_q_aligned(x,y,x_mask,y_mask)
init=tf.global_variables_initializer()
with tf.Session() as s:
    s.run(init)
    print(s.run(result))

In [ ]:
x=np.array([1,2,3,4,5])
y=np.array([1,2,3,4,5])
print(x.shape)

In [ ]:
x=[[[1,2,3],[1,2,3]]]
with tf.Session() as s:
    print(s.run(tf.reduce_sum(x,axis=1)))
    print(s.run(tf.nn.softmax([1,np.inf])))

In [ ]:
x=[1,2]
print(x[1.0])

In [ ]:
print(str(1)+'_'+str(2))

In [ ]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

# Create input data
X = np.random.randn(2, 10, 8)

# The second example is of length 6 
X[1,6,:] = 0
X_lengths = [10, 6]

cell = tf.nn.rnn_cell.LSTMCell(num_units=64, state_is_tuple=True)
cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, output_keep_prob=0.5)
cell = tf.nn.rnn_cell.MultiRNNCell(cells=[cell] * 4, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)

result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=1,
    feed_dict=None)


print(result[0]["outputs"].shape)
print(result[0]["outputs"])
assert result[0]["outputs"].shape == (2, 10, 64)

# Outputs for the second example past past length 6 should be 0
assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()

print(result[0]["last_states"][0].h.shape)
print(result[0]["last_states"][0].h)

In [ ]:
tf.reset_default_graph()

# Create input data
X = np.random.randn(2, 10, 8)

# The second example is of length 6 
X[1,6:] = 0
X_lengths = [10, 6]

cell = tf.nn.rnn_cell.LSTMCell(num_units=64, state_is_tuple=True)

outputs, states  = tf.nn.bidirectional_dynamic_rnn(
    cell_fw=cell,
    cell_bw=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)

output_fw, output_bw = outputs
states_fw, states_bw = states

result = tf.contrib.learn.run_n(
    {"output_fw": output_fw, "output_bw": output_bw, "states_fw": states_fw, "states_bw": states_bw},
    n=1,
    feed_dict=None)

print(result[0]["output_fw"].shape)
print(result[0]["output_bw"].shape)
print(result[0]["states_fw"].h.shape)
print(result[0]["states_bw"].h.shape)

In [ ]:
x=tf.constant([[[1,2,3]],[[1,2,3]]])
y=tf.constant([[1,1,1],[2,2,2]])
print(x.shape)
print(y.shape)
with tf.Session() as s:
    xy=tf.add(x,y)
    print(xy.shape)
    print(s.run(xy))

In [ ]:
i=1
print('sdfs%d' % i)

In [ ]:
import numpy as np  
  
def getPositon():  
    a = np.mat([[2, 5, 7, 8, 9, 89], [6, 7, 5, 4, 6, 90]])  
      
    raw, column = a.shape# get the matrix of a raw and column  
      
    _positon = np.argmax(a)# get the index of max in the a  
    print( _positon )  
    m, n = divmod(_positon, column)  
    print ("The raw is " ,m  )
    print ("The column is ",  n  )
    print ("The max of the a is ", a[m , n])  
  
getPositon() 

In [ ]:
x=[1,2,3]
print(x[1:3])